# INTERPOLATE

by Gan Kuhasubpasin

Interpolate geometry or gpe LDS files to the EA grid using nearest neighbor interpolation.

The code is designed to interpolate up to a 0.25 degree mesh.  
Higher resolutions would be tough to handle for Matlab!

Last Modified: July 15, 2020.

## Import

In [3]:
import numpy as np
import pandas as pd
import math 
import scipy.interpolate
from matplotlib import cm
import matplotlib.mlab as ml
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

import warnings
warnings.simplefilter(action = "ignore", category = FutureWarning)

# display plots in SVG format
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

/Users/boontigan/opt/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## 1. Input section

In [1]:
FileOption = int(input('Enter 1 to load Lithosphere Geometry file or 2 for GPE file  '))

root = input('Enter root name of GEOM or GPE file (root_GEOM/GPE)  ')

res = int(input('Enter choice for grid resolution which geom/gpe values will be interpolated to \nOption 1: 2 degree \nOption 2: 1 degree \nOption 3: 0.5 degree \nOption 4: 0.25 degree ')) 

Enter 1 to load Lithosphere Geometry file or 2 for GPE file  2
Enter root name of GEOM or GPE file (root_GEOM/GPE)  SSA
Enter choice for grid resolution which geom/gpe values will be interpolated to 
Option 1: 2 degree 
Option 2: 1 degree 
Option 3: 0.5 degree 
Option 4: 0.25 degree 1


## 2. Loading geometry/GPE and element/node

In [5]:
#Loading geometry or GPE file
##############################################################################
if FileOption == 1:
    FileLoad = root+'_GEOM'
elif FileOption == 2:
    FileLoad = root+'_GPE'
    
LL = np.loadtxt(FileLoad)

#Load in file with element or node cartesian coordinates --> 
#these files are EA_Elements_resolution.txt or EA_Nodes_resolution.txt.
##############################################################################
if FileOption ==1:
    if res == 1: 
        EA = np.loadtxt('../AbqGrid/EA_Nodes_2.txt', delimiter=',')
    elif res == 2:
        EA = np.loadtxt('../AbqGrid/EA_Nodes_1.txt', delimiter=',')
    elif res == 3:
        EA = np.loadtxt('../AbqGrid/EA_Nodes_0.5.txt', delimiter=',')
    elif res == 4:
        EA = np.loadtxt('../AbqGrid/EA_Nodes_0.25.txt', delimiter=',')
elif FileOption ==2 : 
    if res == 1:
        EA = np.loadtxt('../AbqGrid/EA_Elements_2.txt', delimiter=',')
    elif res == 2:
        EA = np.loadtxt('../AbqGrid/EA_Elements_1.txt', delimiter=',')
    elif res == 3:
        EA = np.loadtxt('../AbqGrid/EA_Elements_0.5.txt', delimiter=',')
    elif res == 4:
        EA = np.loadtxt('../AbqGrid/EA_Elements_0.25.txt', delimiter=',')       

# Create outfile name 
##############################################################################
if FileOption == 1:
    if res == 1:
        outEAfile = root+'_GEOM_EA2N'
    elif res == 2:
        outEAfile = root+'_GEOM_EA1N'
    elif res == 3:
        outEAfile = root+'_GEOM_EA0.5N'
    elif res == 4:
        outEAfile = root+'_GEOM_EA0.25N'

elif FileOption == 2:
    if res == 1:
        outEAfile = root+'_GPE_EA2E'
    elif res == 2:
        outEAfile = root+'_GPE_EA1E'
    elif res == 3:
        outEAfile = root+'_GPE_EA0.5E'
    elif res == 4:
        outEAfile = root+'_GPE_EA0.25E'

# Declare variables
##############################################################################
layers = 1 # number of lithospheric layers
perlayer = len(LL[0]) - 2 # number of data columns to interpolate  -> 3 column
sphPts = len(EA) #value should be 34992 (Elements) or 34992 (nodes)

In [ ]:
len(LL[0])

## 3. Convert grid coordinates

In [6]:
def cart2sph(x,y,z):
    azimuth = np.arctan2(y,x)
    elevation = np.arctan2(z,np.sqrt(x**2 + y**2))
    r = (x**2 + y**2 + z**2)**0.5
    return azimuth, elevation, r

In [7]:
# Convert the EA grid to spherical coordinates
dtor = np.pi/180
EALonRad,EALatRad, EAr = cart2sph(EA[:,1], EA[:,2], EA[:,3])

# Put EA spherical coordinates into new variable and convert the units to degrees
EAsph = np.zeros((sphPts,4))
EAsph[:,0] = EAr
EAsph[:,1] = EALatRad/dtor
EAsph[:,2] = EALonRad/dtor
EAsph[:,3] = EA[:,0]   # Element or node #

## 4. Interpolation
- This process will be taken around 3-4 minutes (due to trigonometry operations in a distance function)

In [8]:
def distance(lat1, long1, lat2, long2):

    # Convert latitude and longitude to spherical coordinates in radians.
    dtor = np.pi/180.0
        
    # phi = 90 - latitude
    phi1 = (90.0 - lat1)*dtor
    phi2 = (90.0 - lat2)*dtor
        
    # theta = longitude
    theta1 = long1*dtor
    theta2 = long2*dtor
        
    # Compute spherical distance from spherical coordinates.     
    # For two locations in spherical coordinates (1, theta, phi) and (1, theta', phi')
    # cosine( arc length ) = sin phi sin phi' cos(theta-theta') + cos phi cos phi'
    # distance = rho * arc length

    arc = np.arccos((np.sin(phi1)*np.sin(phi2)*np.cos(theta1 - theta2) + np.cos(phi1)*np.cos(phi2)))

    return arc/dtor

In [9]:
#Distance array
dt = np.zeros(len(LL))

#Output array 
LL_itp = np.zeros((sphPts,perlayer+3))
LL_itp[:,0] = EAsph[:,3]     #1st column: number of grid point
LL_itp[:,1:3] = EAsph[:,1:3] #2nd 3rd column: lat and lon

for i in range(0,sphPts):
    
    # finding the distance between two points on a sphere (in degrees!)
    dt = distance(EAsph[i,1], EAsph[i,2], LL[:,0], LL[:,1])
                  
    # Sorting "dt" so that the distances are listed in ascending order
    dts = np.argsort(dt)
    
    # Finding the 4 closest points to the node or element grid point 
    # and then find the value associated with each point.   
    value = np.zeros((4,perlayer))  
    npd = np.zeros(4)
    for j in range (0,4):
        npd[j] = dt[dts[j]]
        value[j,0:perlayer] = LL[dts[j],2:perlayer+2]

    # Calculating and printing out all of the interpolated values
    # Interpolate the values using a simple weighted mean algorithim: 
    # multiplying each value by the distance (weigths) to the element point 
    # and then dividing by the sum of the distances
    for k in range(0,layers*perlayer):
        LL_itp[i,k+3]= sum(value[:,k]*npd)/(sum(npd))

## 5. Write output file

In [10]:
fmt = '%.1f\t%.4f\t%.4f\t%.4f\t%.4f\t%.4f'
np.savetxt(outEAfile, LL_itp, delimiter=' ',fmt=fmt)